# PFE : Analyse de données statistiques sportives

## I. Scrapping des données

Dans un premier temps, nous allons chercher à prédire le prochain MVP en NBA.
Pour cela, on s'intéressera aux informations suivantes :
- Votes MVP de chaque année depuis 1980 
- Statistiques des joueurs depuis 1980
- Statistiques des équipes depuis 1980  

Remarque : On récupère les données depuis 1980 car la saison 1980-1981 était la première saison pour laquelle ce n'était plus les joueurs qui votaient mais des journalistes et des commentateurs comme c'est le cas actuellement.

### Basketball Reference
Nous récupérons les données (dans un premier temps) sur le site https://www.basketball-reference.com qui répertorie un nombre conséquent de statistiques en NBA pour chaque saison.


On commence tout d'abord par repérer les pages qui vont nous intéresser

### Votes MVP

<img src="image/img1.png" style="height:300px">

Sur cette page, on a les votes du MVP pour la saison 2021-2022 ainsi il faudrait être capable de récupérer la page correspondante pour chaque saison depuis 1980.
Ce qui nous fait une quarantaine de page à récupérer

### Statistiques joueurs

<img src="image/img2.png" style="height:300px">

Sur cette page, on a les statistiques moyennes par match pour chaque joueur pour la saison 2021-2022 ainsi il faudrait être capable de récupérer la page correspondante pour chaque saison depuis 1980. Ce qui nous fait une quarantaine de page à récupérer.

Remarque : Certains joueurs apparaissent en double voire triple car ces statistiques sont calculés pour chaque équipe
dans lesquelles ils ont joué durant la saison

### Statistiques des équipes

<img src="image/img3.png" style="height:300px">

Sur cette page, nous avons les statistiques des équipes de la conférence Est de la NBA ainsi que celles de la conférence Ouest.
On va chercher à récupérer une combinaison de ces deux tableaux pour chaque année depuis 1980. Ce qui nous fait une quarantaine de page à récupérer.

### BILAN
On a donc environ 120 pages à récupérer, il va donc nous falloir 120 requêtes.

### PROBLEME

Le site Basketball Reference possède une protection contre les attaques DDOS ce qui provoque un ban ip au bout d'une trantaine de requêtes consécutives, ce qui pose problème dans notre cas.

On a donc trouvé une solution en nous tournant vers le module Selenium qui de part son fonctionnement, contourne cette protection.

## CODE

### Import des librairies nécessaires

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time
import sqlite3

### Définition de variable générale

In [4]:
years =  list(range(1980,2023)) #liste des années dont on récupère les données
s = Service('/chromedriver/chromedriver.exe')
driver = webdriver.Chrome(service = s) #définition du driver chrome pour le scrapping avec selenium

### A) Scrapping votes MVP

1) Récupération des fichiers html associés à chaque page contenant les votes MVP que l'on stocke dans un dossier "mvp" 

In [ ]:
url_mvps = "https://www.basketball-reference.com/awards/awards_{}.html"
for year in years:
    url = url_mvps.format(year)
    driver.get(url)
    html = driver.page_source #grab the data
    with open("mvp/{}.html".format(year),"w+",encoding="utf-8") as f:
        f.write(html)
    

2) Exploitation des fichiers html pour créer un dataframe exploitable

En s'intéressant au fichier html, en repère rapidement la partie du code qui va nous intéresser (ici la balise ayant pour id  "mvp")

<img src="image/img4.png" style="height:400px">

On en profite pour retirer directement la ligne "over_header" qui n'a pas d'utilité dans notre étude 

<img src="image/img5.png" style="height:400px">

In [ ]:
dfs = []
for year in years:
    with open("mvp/{}.html".format(year),encoding="utf-8") as f:
        page = f.read()
    soup = BeautifulSoup(page,"html.parser")
    if(soup.find('tr',class_ = "over_header")!=None):
        soup.find('tr',class_ = "over_header").decompose()
    
    mvp_table = soup.find_all(id = "mvp")
    mvp = pd.read_html(str(mvp_table))[0]
    mvp["Year"] = year #Pour distinguer l'année dont vient les données
    dfs.append(mvp)
mvps = pd.concat(dfs)
mvps

3) Envoi dans un base de donnée SQL

In [ ]:
mvpcon = sqlite3.connect('mvp.sqlite')

In [ ]:
mvps.to_sql("MVP", con = mvpcon)

In [ ]:
print(mvpcon.execute("SELECT * FROM MVP").fetchall())

### B)  Scrapping statistiques joueurs

1) Récupération des fichiers html associés à chaque page contenant les statistiques des joueurs que l'on stocke dans un dossier "player"

In [ ]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
for year in years:
    url = player_stats_url.format(year)
    driver.get(url)
    html = driver.page_source #grab the data
    with open("player/{}.html".format(year),"w+",encoding="utf-8") as f:
        f.write(html)
  

In [7]:
player_advanced_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_advanced.html"
for year in years:
    url = player_advanced_stats_url.format(year)
    driver.get(url)
    html = driver.page_source #grab the data
    with open("advanced/{}.html".format(year),"w+",encoding="utf-8") as f:
        f.write(html)

2) Exploitation des fichiers html pour créer un dataframe exploitable

En s'intéressant au fichier html, en repère rapidement la partie du code qui va nous intéresser (ici la balise ayant pour id  "per_game_stats")

<img src="image/img8.png" style="height:400px">

On en profite pour retirer les "thead" qui ne sont pas utiles pour notre étude

<img src="image/img9.png" style="height:400px">

In [ ]:
dfs2 = []
for year in years:
    with open("player/{}.html".format(year),encoding="utf-8") as f:
        page = f.read()
    soup = BeautifulSoup(page,"html.parser")
    tHeads = soup.findAll('tr', class_="thead")
    for tHead in tHeads:
        tHead.decompose()
    player_table = soup.find(id = "per_game_stats")
    player = pd.read_html(str(player_table))[0]
    player["Year"] = year
    dfs2.append(player)
players = pd.concat(dfs2)
players

In [10]:
dfs2_bis = []
for year in years:
    with open("advanced/{}.html".format(year),encoding="utf-8") as f:
        page = f.read()
    soup = BeautifulSoup(page,"html.parser")
    tHeads = soup.findAll('tr', class_="thead")
    for tHead in tHeads:
        tHead.decompose()
    advanced_table = soup.find(id = "advanced_stats")
    advanced = pd.read_html(str(advanced_table))[0]
    advanced["Year"] = year
    dfs2_bis.append(advanced)
advancedf = pd.concat(dfs2_bis)
advancedf

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Year
0,1,Kareem Abdul-Jabbar*,C,32,LAL,82,3143,25.3,0.639,0.001,...,9.5,5.3,14.8,0.227,NaN,4.8,2.4,7.2,7.3,1980
1,2,Tom Abernethy,PF,25,GSW,67,1222,11.0,0.511,0.003,...,1.2,0.8,2.0,0.080,NaN,-1.0,-0.2,-1.2,0.2,1980
2,3,Alvan Adams,C,25,PHO,75,2168,19.2,0.571,0.002,...,3.1,3.9,7.0,0.155,NaN,1.7,1.9,3.6,3.1,1980
3,4,Tiny Archibald*,PG,31,BOS,80,2864,15.3,0.574,0.023,...,5.9,2.9,8.9,0.148,NaN,1.4,-0.3,1.1,2.3,1980
4,5,Dennis Awtrey,C,31,CHI,26,560,7.4,0.524,0.000,...,0.1,0.5,0.6,0.053,NaN,-2.3,0.9,-1.4,0.1,1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,535,Thaddeus Young,PF,34,TOR,54,795,14.1,0.573,0.172,...,0.7,1.1,1.8,0.109,NaN,-1.8,1.9,0.1,0.4,2023
675,536,Trae Young,PG,24,ATL,73,2541,22.0,0.573,0.331,...,5.3,1.4,6.7,0.126,NaN,5.3,-2.0,3.3,3.4,2023
676,537,Omer Yurtseven,C,24,MIA,9,83,16.7,0.675,0.259,...,0.2,0.1,0.3,0.159,NaN,-2.5,-1.5,-3.9,0.0,2023
677,538,Cody Zeller,C,30,MIA,15,217,16.4,0.659,0.034,...,0.4,0.3,0.7,0.147,NaN,-2.0,-0.7,-2.8,0.0,2023


In [13]:
del advancedf['Unnamed: 19']
del advancedf['Unnamed: 24']

3) Envoi dans une base de donnée SQL

In [14]:
advancedcon = sqlite3.connect('advanced_stats.sqlite')

In [ ]:
playerscon = sqlite3.connect('players.sqlite')

In [15]:
advancedf.to_sql("ADVANCED_STATS", con = advancedcon)

23146

In [ ]:
players.to_sql("PLAYERS", con = playerscon)

### C) Scrapping des statistiques des équipes

1) Récupération des fichiers html associés à chaque page contenant les statistiques des équipes que l'on stocke dans un dossier "team"

In [ ]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"
for year in years:
    url = team_stats_url.format(year)
    driver.get(url)
    html = driver.page_source #grab the data
    with open("team/{}.html".format(year),"w+",encoding="utf-8") as f:
        f.write(html)

2) Exploitation des fichiers html pour créer un dataframe exploitable

En s'intéressant au fichier html, en repère rapidement la partie du code qui va nous intéresser. Ici ce sont les balises ayant pour id  "divs_standings_E" et "divs_standings_W" qui représentent respectivement les statistiques des équipes de la conférence Est de la Nba et celle de l'Ouest

Conférence Est
<img src="image/img10.png" style="height:350px">
Conférence Ouest
<img src="image/img11.png" style="height:350px">

On en profite pour retirer encore une fois les "thead" inutiles

<img src="image/img12.png" style="height:300px">

In [ ]:
dfs3 = []
for year in years: 
    
    with open("team/{}.html".format(year),encoding="utf-8") as f:
        page = f.read()
    soup = BeautifulSoup(page,"html.parser")
    if(soup.find('tr', class_="thead")!=None):
         soup.find('tr', class_="thead").decompose()
    team_table = soup.find(id = "divs_standings_E")
    team = pd.read_html(str(team_table))[0]
    team["Year"] = year
    team["Team"] = team["Eastern Conference"]
    del team["Eastern Conference"]
    dfs3.append(team)

    soup = BeautifulSoup(page,"html.parser")
    if(soup.find('tr', class_="thead")!=None):
        
        soup.find('tr', class_="thead").decompose()
   
    team_table = soup.find(id = "divs_standings_W")
    team = pd.read_html(str(team_table))[0]
    team["Year"] = year
    team["Team"] = team["Western Conference"]
    del team["Western Conference"]
    dfs3.append(team)
teams = pd.concat(dfs3)
teams

3) Envoi dans un base de donnée SQL

In [ ]:
teamscon = sqlite3.connect('teams.sqlite')

In [ ]:
teams.to_sql("TEAMS", con = teamscon)